In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# evaluate accuracy of the original networks before finetuning
from onnx2torch import convert
from stitchnet.stitchonnx.utils import accuracy_score_net,Net,get_score_net,load_cats_and_dogs_dset,load_dl,eval_original_model,get_macs_params_onnx,get_data_score

from stitchnet.stitchonnx.viz import draw_stitchNet_fromTuples
import torch
import os
from glob import glob
from tqdm import tqdm
from skl2onnx.helpers.onnx_helper import load_onnx_model
from pathlib import Path
data_score = get_data_score(32)

batch_size = 32
dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")
dataloaders = dict(
    train=load_dl(dataset_train, batch_size, shuffle=True),
    val=load_dl(dataset_val, batch_size, shuffle=False)
)
os.makedirs('_results/original', exist_ok=True)

modelnames = sorted(glob('_models/*.onnx'))
for i,modelname in tqdm(enumerate(modelnames)):
    name = os.path.basename(modelname)
    namewithoutext = os.path.splitext(name)[0]
    # if os.path.exists(f'_results/original/{namewithoutext}.txt'):
    #     continue
    model_onnx1 = load_onnx_model(modelname)
    macs, params = get_macs_params_onnx(model_onnx1)
    torch_model_1 = convert(model_onnx1)
    valacc,trainacc = eval_original_model(torch_model_1, dataloaders)
    
    # get score
    fragmentFiles = sorted(glob(f'_models/fragments/net{i:03}/*.onnx'))
    onnxFragments = []
    js = []
    for j,fragmentFile in enumerate(fragmentFiles):
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
        js.append((i,j))
    net1 = Net(onnxFragments, i)
    score = get_score_net(net1, data_score)
    
    accuracy = accuracy_score_net(Net([model_onnx1]), dataset_val, bs=256)
    print('ACC:',accuracy, valacc, trainacc)
    
    with open(f'_results/original/{namewithoutext}.txt', 'w') as f:
        f.write(f'{valacc},{trainacc},{macs},{params},{score},"{tuple(js)}"\n')
        
    draw_stitchNet_fromTuples(js, name=f'_results/original/{namewithoutext}')
        # f.write(f'{",".join([f"{x:.4}" for x in [valacc,trainacc,macs,params,score]])}\n')
    # break

0it [00:00, ?it/s]
100%|██████████| 623/623 [00:20<00:00, 30.81it/s]

100%|██████████| 160/160 [00:05<00:00, 26.69it/s]

0it [00:00, ?it/s]
1it [00:01,  1.72s/it]
2it [00:01,  1.30it/s]

input.4 input.4 0.9999999403953552
input.12 input.12 1.0



3it [00:01,  2.09it/s]
4it [00:02,  2.93it/s]

onnx::Conv_24 onnx::Conv_24 1.0000001192092896
onnx::Conv_26 onnx::Conv_26 0.9999999403953552



5it [00:04,  1.12s/it]

...

onnx::Gemm_67 onnx::Gemm_67 0.9679515361785889



15it [00:30,  2.02s/it]

onnx::Gemm_69 onnx::Gemm_69 0.9776305556297302




  0%|          | 0/20 [00:00<?, ?it/s]2022-08-14 12:47:15.810572497 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

  5%|▌         | 1/20 [00:06<01:55,  6.07s/it]2022-08-14 12:47:16.081471769 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

 10%|█         | 2/20 [00:06<00:52,  2.91s/it]2022-08-14 12:47:16.587814463 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

 15%|█▌        | 3/20 [00:07<00:30,  1.81s/it]2022-08-14 12:47:17.086851281 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

 20%|██        | 4/20 [00:07<00:20,  1.29s/it]2022-08-14 12:47:17.588701154 [W:onnxrun

ACC: 0.8115913555992141 0.8117878192534381 0.8035660472124561


In [2]:
from stitchnet.stitchonnx.utils import load_dl,load_onnx_model,load_cats_and_dogs_dset,execute_fragments,convert_imagenet_to_cat_dog_label
import numpy as np
from glob import glob
from tqdm import tqdm

dataset_val = load_cats_and_dogs_dset("test")
fragmentFiles = sorted(glob(f'_models/fragments/net001/*.onnx'))
onnxFragments = []
js = []
for j,fragmentFile in enumerate(fragmentFiles):
    onnxFragment = load_onnx_model(fragmentFile)
    onnxFragments.append(onnxFragment)

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
